In [1]:
import spacy
import csv
from spacy import displacy

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
# with open('devset3_claude.csv') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         orig = row['Original Sentence']
#         new = row['New Sentence']

#         print(orig, new)
        
#         doc1 = nlp(orig)
#         doc2 = nlp(new)
#         displacy.serve([doc1, doc2], style="dep", auto_select_port=True)

#         input()

In [4]:
import statistics
import collections
import tqdm
all_uuas = []
matches = collections.defaultdict(lambda: [0, 0])
with open('devset3_claude.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in tqdm.tqdm(reader):
        orig = row['Original Sentence']
        new = row['New Sentence']
        
        doc1 = nlp(orig)
        doc2 = nlp(new)


        a = [i.head.i for i in doc1[:-1]]
        b = [i.head.i for i in doc2[:-1]]
        if len(a) != len(b):
            continue

        
        uuas = sum(1 for i in range(len(a)) if a[i] == b[i]) / len(a)

        for idx, (i, j) in enumerate(zip(doc1[:-1], doc2[:-1])):
            if a[idx] == b[idx] and i.dep_ == j.dep_:
                matches[i.dep_][0] += 1
            matches[i.dep_][1] += 1

        all_uuas.append(uuas)

print(statistics.mean(all_uuas))

102it [00:00, 124.57it/s]

0.862735401782077


In [5]:
for k, v in sorted(matches.items(), key = lambda a: -(a[1][0] + a[1][1])):
    print(f"{k.ljust(10)}: {round(v[0] / v[1] * 100, 2)}% ({v[0]}/{v[1]})")

pobj      : 86.75% (72/83)
prep      : 72.41% (63/87)
det       : 91.03% (71/78)
nsubj     : 87.84% (65/74)
compound  : 86.44% (51/59)
punct     : 66.67% (38/57)
ROOT      : 84.0% (42/50)
aux       : 92.11% (35/38)
advmod    : 71.79% (28/39)
dobj      : 65.0% (26/40)
amod      : 64.52% (20/31)
cc        : 62.5% (15/24)
nummod    : 94.44% (17/18)
conj      : 61.9% (13/21)
ccomp     : 76.47% (13/17)
poss      : 87.5% (14/16)
npadvmod  : 50.0% (9/18)
mark      : 100.0% (11/11)
case      : 90.91% (10/11)
nmod      : 58.33% (7/12)
advcl     : 60.0% (6/10)
attr      : 20.0% (2/10)
auxpass   : 71.43% (5/7)
nsubjpass : 83.33% (5/6)
neg       : 66.67% (4/6)
xcomp     : 66.67% (4/6)
acl       : 33.33% (2/6)
relcl     : 60.0% (3/5)
quantmod  : 75.0% (3/4)
pcomp     : 66.67% (2/3)
acomp     : 66.67% (2/3)
agent     : 100.0% (2/2)
appos     : 100.0% (2/2)
dative    : 0.0% (0/2)
expl      : 0.0% (0/1)
intj      : 0.0% (0/1)
parataxis : 0.0% (0/1)
prt       : 0.0% (0/1)


In [31]:
from tabulate import tabulate
buckets = []

for bucket in range(100, 0, -5):
    buckets.append(f"{bucket-5}-{bucket}%")
#     n = sum(1 for i in all_uuas if bucket - 5 <= i * 100 <= bucket)
#     uuass.append(f"{round(n / len(all_uuas) * 100, 2)}%")

#     matches = collections.defaultdict(lambda: [0, 0])
#     for idx, (i, j) in enumerate(zip(doc1[:-1], doc2[:-1])):
#             if a[idx] == b[idx] and i.dep_ == j.dep_:
#                 matches[i.dep_][0] += 1
#             matches[i.dep_][1] += 1
uuass = [0 for i in buckets]
matches = [collections.defaultdict(lambda: [0, 0]) for i in buckets]
with open('devset3_claude.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in tqdm.tqdm(reader):
        orig = row['Original Sentence']
        new = row['New Sentence']
        
        doc1 = nlp(orig)
        doc2 = nlp(new)


        a = [i.head.i for i in doc1[:-1]]
        b = [i.head.i for i in doc2[:-1]]
        if len(a) != len(b):
            continue

        
        uuas = sum(1 for i in range(len(a)) if a[i] == b[i]) / len(a)
        
        bucket = int(20 - ((uuas - 0.0001) * 100 // 5) - 1)
        # print(uuas, bucket)

        uuass[bucket] += 1
        
        
        for idx, (i, j) in enumerate(zip(doc1[:-1], doc2[:-1])):
            if a[idx] == b[idx] and i.dep_ == j.dep_:
                matches[bucket][i.dep_][0] += 1
            matches[bucket][i.dep_][1] += 1

uuass = [f"{round(i / sum(uuass) * 100, 2)}%" for i in uuass]



102it [00:00, 141.93it/s]

+---------+------------+
| Bucket  | Proportion |
+---------+------------+
| 95-100% |   44.0%    |
| 90-95%  |   10.0%    |
| 85-90%  |    4.0%    |
| 80-85%  |   14.0%    |
| 75-80%  |    6.0%    |
| 70-75%  |    4.0%    |
| 65-70%  |    8.0%    |
| 60-65%  |    2.0%    |
| 55-60%  |    2.0%    |
| 50-55%  |    2.0%    |
| 45-50%  |    2.0%    |
| 40-45%  |    0.0%    |
| 35-40%  |    0.0%    |
| 30-35%  |    2.0%    |
| 25-30%  |    0.0%    |
| 20-25%  |    0.0%    |
| 15-20%  |    0.0%    |
| 10-15%  |    0.0%    |
|  5-10%  |    0.0%    |
|  0-5%   |    0.0%    |
+---------+------------+


In [35]:
x = [i.split(":")[0].strip() for i in """pobj      : 86.75% (72/83)
prep      : 72.41% (63/87)
det       : 91.03% (71/78)
nsubj     : 87.84% (65/74)
compound  : 86.44% (51/59)
punct     : 66.67% (38/57)
ROOT      : 84.0% (42/50)
aux       : 92.11% (35/38)
advmod    : 71.79% (28/39)
dobj      : 65.0% (26/40)
amod      : 64.52% (20/31)
cc        : 62.5% (15/24)
nummod    : 94.44% (17/18)
conj      : 61.9% (13/21)
ccomp     : 76.47% (13/17)
poss      : 87.5% (14/16)
npadvmod  : 50.0% (9/18)
mark      : 100.0% (11/11)
case      : 90.91% (10/11)
nmod      : 58.33% (7/12)
advcl     : 60.0% (6/10)
attr      : 20.0% (2/10)
auxpass   : 71.43% (5/7)
nsubjpass : 83.33% (5/6)
neg       : 66.67% (4/6)
xcomp     : 66.67% (4/6)
acl       : 33.33% (2/6)
relcl     : 60.0% (3/5)
quantmod  : 75.0% (3/4)
pcomp     : 66.67% (2/3)
acomp     : 66.67% (2/3)
agent     : 100.0% (2/2)
appos     : 100.0% (2/2)
dative    : 0.0% (0/2)
expl      : 0.0% (0/1)
intj      : 0.0% (0/1)
parataxis : 0.0% (0/1)
prt       : 0.0% (0/1)""".split("\n")]

In [37]:
deps = collections.defaultdict(lambda: [0 for i in buckets])
for i, m in enumerate(matches):
    for k, v in m.items():
        deps[k][i] = f"{round(v[0] / v[1] * 100, 2)}%"

defaultdict(<function __main__.<lambda>()>,
            {'compound': ['96.0%',
              '81.82%',
              '100.0%',
              '75.0%',
              '66.67%',
              '100.0%',
              '57.14%',
              '100.0%',
              '100.0%',
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
             'nsubj': ['100.0%',
              '75.0%',
              '80.0%',
              '91.67%',
              '100.0%',
              '100.0%',
              '80.0%',
              '50.0%',
              '0.0%',
              '100.0%',
              '75.0%',
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
             'ROOT': ['100.0%',
              '100.0%',
              '100.0%',
              '71.43%',
              '100

In [56]:
o = [list(deps.keys()).index(i) for i in x]
v = [list(deps.values())[i] for i in o][:10]
k = [list(deps.keys())[i] for i in o][:10]

print(tabulate(zip(buckets, uuass, *v), tablefmt="pretty", headers=["Bucket", "Proportion"] + k))

+---------+------------+--------+--------+--------+--------+----------+--------+--------+--------+--------+--------+
| Bucket  | Proportion |  pobj  |  prep  |  det   | nsubj  | compound | punct  |  ROOT  |  aux   | advmod |  dobj  |
+---------+------------+--------+--------+--------+--------+----------+--------+--------+--------+--------+--------+
| 95-100% |   44.0%    | 100.0% | 100.0% | 100.0% | 100.0% |  96.0%   | 100.0% | 100.0% | 100.0% | 100.0% | 81.82% |
| 90-95%  |   10.0%    | 100.0% | 75.0%  | 87.5%  | 75.0%  |  81.82%  | 88.89% | 100.0% | 100.0% | 100.0% | 75.0%  |
| 85-90%  |    4.0%    | 100.0% | 100.0% | 100.0% | 80.0%  |  100.0%  |  0.0%  | 100.0% | 100.0% | 100.0% |   0    |
| 80-85%  |   14.0%    | 91.67% | 50.0%  | 93.75% | 91.67% |  75.0%   | 78.57% | 71.43% | 100.0% | 58.33% | 75.0%  |
| 75-80%  |    6.0%    | 100.0% | 66.67% | 100.0% | 100.0% |  66.67%  | 100.0% | 100.0% | 100.0% | 33.33% |   0    |
| 70-75%  |    4.0%    | 100.0% | 75.0%  | 83.33% | 100.0% |  10